# Part 4: The Showdown (Evaluation Arena)

Which model is better? Cost vs. Accuracy.

**Goal:** Run Golden Test Set through BOTH models (Fine-tuned vs RAG).

**Metrics:**
1. **ROUGE-L**: Textual overlap with ground truth.
2. **LLM-as-a-Judge**: "Faithfulness" and "Accuracy" (1-5 scale).
3. **Latency**: Time to generate response (ms).

**Bonus:** Cost Comparison (500 daily users * 10 queries).

In [2]:
import os
import sys
import time
import json
import pandas as pd
from tqdm.auto import tqdm
from pathlib import Path
from dotenv import load_dotenv
from rouge_score import rouge_scorer
from typing import List, Dict, Any

# Setup paths
notebook_dir = Path.cwd()
if notebook_dir.name == "notebooks":
    project_root = notebook_dir.parent
else:
    project_root = notebook_dir

os.chdir(project_root)
sys.path.insert(0, str(project_root))

print(f"📂 Working directory: {os.getcwd()}")

# Load environment variables
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Load config
from src.services.llm_services import load_config
config = load_config("src/config/config.yaml")

📂 Working directory: d:\Bootcamps\AEE 2026 Jan\Mini Projects\Operation Ledger Mind


## 1. Load Golden Test Set

In [3]:
def load_jsonl(filepath: str) -> List[Dict]:
    data = []
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))
    return data

test_file = Path(config["output_dir"]) / "golden_test_set.jsonl"
test_set = load_jsonl(test_file)

print(f"✅ Loaded {len(test_set)} test cases from {test_file}")
if test_set:
    print(f"Sample: {test_set[0]['question']}")

✅ Loaded 240 test cases from data\processed\golden_test_set.jsonl
Sample: What can be inferred about the future relationship between Uber and its Drivers based on the current challenges mentioned?


## 2. Setup RAG System

In [4]:
import weaviate
from weaviate.connect import ConnectionParams
from sentence_transformers import CrossEncoder
from openai import OpenAI
from collections import defaultdict

# Weaviate Client
client = weaviate.WeaviateClient(
    connection_params=ConnectionParams.from_url("http://localhost:8080", grpc_port=50051)
)
client.connect()
col = client.collections.get("FinancialChunk")

# Reranker
reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L6-v2")

# OpenAI Client
oa = OpenAI(api_key=OPENAI_API_KEY)

# RAG Functions
def retrieve_bm25(q: str, k: int = 25):
    res = col.query.bm25(query=q, limit=k)
    return res.objects

def retrieve_vector(q: str, k: int = 25):
    res = col.query.near_text(query=q, limit=k)
    return res.objects

def rrf_fuse(lists, rrf_k=60, top_n=30):
    scores = defaultdict(float)
    for lst in lists:
        for rank, obj in enumerate(lst, start=1):
            scores[str(obj.uuid)] += 1.0 / (rrf_k + rank)
    ranked_ids = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:top_n]
    wanted = {rid for rid, _ in ranked_ids}
    by_id = {}
    for lst in lists:
        for obj in lst:
            if str(obj.uuid) in wanted and str(obj.uuid) not in by_id:
                by_id[str(obj.uuid)] = obj
    return [by_id[rid] for rid, _ in ranked_ids if rid in by_id]

def cross_encoder_rerank(question, candidates, top_n=8):
    pairs = [(question, c.properties["text"]) for c in candidates]
    if not pairs:
        return []
    scores = reranker.predict(pairs)
    ranked = sorted(zip(candidates, scores), key=lambda x: x[1], reverse=True)
    return [c for c, _ in ranked[:top_n]]

def build_context(objs):
    out = []
    for i, o in enumerate(objs, start=1):
        out.append(f"[{i}] (p.{o.properties.get('page')})\n{o.properties.get('text')}")
    return "\n\n".join(out)

def query_rag(question: str) -> Dict[str, Any]:
    t0 = time.time()
    bm25 = retrieve_bm25(question)
    vec = retrieve_vector(question)
    fused = rrf_fuse([bm25, vec])
    top = cross_encoder_rerank(question, fused)
    context = build_context(top)
    
    system = "You are a financial assistant. Answer using the provided context."
    user_msg = f"Context:\n{context}\n\nQuestion: {question}\nAnswer:"
    
    resp = oa.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": system}, {"role": "user", "content": user_msg}],
        temperature=0.2
    )
    latency = int((time.time() - t0) * 1000)
    return {"answer": resp.choices[0].message.content, "latency": latency}

## 3. Setup Fine-Tuned Model

The setup of the fine tuned model and the running of the main evaluation loop for it are done in the setup_ft.ipynb file. FT results are saved in generated data folder.

## 4. Evaluation Metrics (ROUGE + LLM Judge)

In [5]:
rouge = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

def calculate_rouge(pred: str, ref: str) -> float:
    scores = rouge.score(ref, pred)
    return scores['rougeL'].fmeasure

def llm_judge(question: str, ground_truth: str, prediction: str) -> int:
    if not prediction or not prediction.strip():
        return 1
        
    prompt = f"""You are an impartial judge evaluating the quality of an AI-generated answer.
    
    Question: {question}
    Ground Truth: {ground_truth}
    Prediction: {prediction}
    
    Rate the Prediction on a scale of 1 to 5 based on Faithfulness and Accuracy compared to the Ground Truth.
    1 = Completely wrong or irrelevant
    3 = Partially correct but missing details
    5 = Perfect match in information and intent
    
    Return ONLY the integer score (1-5)."""
    
    try:
        resp = oa.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0
        )
        score = int(resp.choices[0].message.content.strip())
        return max(1, min(5, score))
    except:
        return 1

## 5. Main Evaluation Loop

In [6]:
import pandas as pd
from tqdm import tqdm

results = []
limit = 50

ENABLE_JUDGE = True

print(f"🚀 Starting evaluation on {min(len(test_set), limit)} samples...")

for i, item in enumerate(tqdm(test_set[:limit], desc="Eval")):
    q = item["question"]
    gt = item["answer"]
    category = item.get("category", "Unknown")

    row = {"question": q, "category": category, "original_answer": gt}

    # --- RAG ---
    try:
        rag_res = query_rag(q)
        rag_ans = rag_res["answer"]
        rag_lat = rag_res["latency"]
        rag_rouge = calculate_rouge(rag_ans, gt)
        rag_score = llm_judge(q, gt, rag_ans) if ENABLE_JUDGE else None
    except Exception as e:
        rag_ans, rag_lat, rag_rouge, rag_score = "ERROR", None, None, None

    row.update({
        "RAG_Answer": rag_ans,
        "RAG_Latency": rag_lat,
        "RAG_ROUGE": rag_rouge,
        "RAG_Score": rag_score
    })

    results.append(row)

    # Save partial every 5 rows
    if (i + 1) % 5 == 0:
        pd.DataFrame(results).to_csv("data/generated/eval_rag_partial.csv", index=False)

df = pd.DataFrame(results)
df.to_csv("data/generated/eval_rag_results.csv", index=False)
print("✅ Evaluation Complete! Saved: eval_rag_results.csv")


🚀 Starting evaluation on 50 samples...


Eval: 100%|██████████| 50/50 [07:18<00:00,  8.78s/it]

✅ Evaluation Complete! Saved: eval_rag_results.csv


## 6. Results Analysis

In [7]:
# Load results
rag_results_path = "data/generated/eval_rag_results.csv"
ft_results_path = "data/generated/eval_ft_results.csv"

if os.path.exists(rag_results_path) and os.path.exists(ft_results_path):
    print(f"✅ Loading results from:\n - {rag_results_path}\n - {ft_results_path}")
    rag_df = pd.read_csv(rag_results_path)
    ft_df = pd.read_csv(ft_results_path)

    # Merge dataframes
    # Select columns from FT to avoid duplicates
    ft_cols = ['question', 'FT_Latency', 'FT_ROUGE', 'FT_Score']
    if 'FT_Answer' in ft_df.columns: ft_cols.append('FT_Answer')
    
    df = pd.merge(rag_df, ft_df[ft_cols], on='question', how='inner')
    
    if not df.empty:
        # Average Metrics
        metrics = ["RAG_Latency", "RAG_ROUGE", "RAG_Score", "FT_Latency", "FT_ROUGE", "FT_Score"]
        existing = [m for m in metrics if m in df.columns]
        
        print("📊 Overall Summary:")
        print(df[existing].mean())

        # Check if we have valid scores
        has_rag_score = "RAG_Score" in df.columns and df["RAG_Score"].notna().any()
        has_ft_score = "FT_Score" in df.columns and df["FT_Score"].notna().any()

        if has_rag_score or has_ft_score:
            cols = []
            if has_rag_score: cols.append("RAG_Score")
            if has_ft_score: cols.append("FT_Score")
            
            print("\n📊 By Category (Score):")
            print(df.groupby("category")[cols].mean())
        else:
             print("\n⚠️ No LLM Judge Scores found (did you enable ENABLE_JUDGE?). Comparison by ROUGE:")
             print(df.groupby("category")[["RAG_ROUGE", "FT_ROUGE"]].mean())
else:
    print("❌ Results files not found.")

✅ Loading results from:
 - data/generated/eval_rag_results.csv
 - data/generated/eval_ft_results.csv
📊 Overall Summary:
RAG_Latency     7945.940000
RAG_ROUGE          0.292667
RAG_Score          4.540000
FT_Latency     12567.660000
FT_ROUGE           0.303838
FT_Score           1.000000
dtype: float64

📊 By Category (Score):
                    RAG_Score  FT_Score
category                               
Hard Facts           4.291667       1.0
Strategic Summary    5.000000       1.0
Stylistic/Creative   4.625000       1.0


## 7. Cost Analysis (Bonus)
**Scenario:** 500 users * 10 queries/day = 5,000 queries/day = 150,000 queries/month.

**RAG (GPT-4o-mini):**
- Input: ~1500 tokens (context) + 50 tokens (query) = 1550 tokens
- Output: ~200 tokens
- Price: $0.15/1M in, $0.60/1M out

**Fine-Tuned (Self-Hosted GPU):**
- Needs AWS g5.xlarge (A10G) -> ~$1.01/hr
- Running 24/7

In [8]:
queries_per_month = 150000
tokens_in = 1550
tokens_out = 200

# RAG Cost
cost_rag_in = (queries_per_month * tokens_in / 1_000_000) * 0.15
cost_rag_out = (queries_per_month * tokens_out / 1_000_000) * 0.60
total_rag = cost_rag_in + cost_rag_out

# FT Cost (AWS g5.xlarge on-demand)
hourly_rate = 1.01
total_ft = hourly_rate * 24 * 30

print(f"💰 Monthly Cost Comparison (150k queries):")
print(f"   RAG (GPT-4o-mini): ${total_rag:.2f}")
print(f"   Fine-Tuned (g5.xlarge): ${total_ft:.2f}")
print(f"\n   Winner for Cost: {'RAG' if total_rag < total_ft else 'Fine-Tuned'}")

💰 Monthly Cost Comparison (150k queries):
   RAG (GPT-4o-mini): $52.88
   Fine-Tuned (g5.xlarge): $727.20

   Winner for Cost: RAG


In [9]:
df.to_csv("data/final/final_results.csv", index=False)
print("✅ Evaluation Complete! Saved: final_results.csv")

✅ Evaluation Complete! Saved: final_results.csv
